In [16]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import joblib
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [17]:
def data_cleaning_for_regression(df):

    # Replacing missing values

    df=df.replace(999, np.nan)
    imputer = joblib.load('imputer_model.joblib')

    df_imputed=imputer.transform(df)

    df_imputed = pd.DataFrame(df_imputed, columns=df.columns)

    # Dealing with Categorical variables

    #  Age,ChemoGrade,Proliferation and TumorStage are ordinal categorical variables so we do not change them

    # ER, PGR, HER2, TrippleNegative, HistologyType and LNStatus are either binary or categorical variables with 2 different
    # values. We just convert HistologyType to binary format.

    df_imputed['HistologyType'] = df_imputed['HistologyType'].map({1: 0, 2: 1})

    return df_imputed

In [18]:
test_data=pd.read_excel('TestDatasetExample.xls')
test_df_imputed = data_cleaning_for_regression(test_data.drop('ID',axis=1))

In [19]:
test_X = test_df_imputed.iloc[:, 1:]
test_y = test_df_imputed.iloc[:, 0]

# Feature Selection based on AnovaThreshold 3
features = ['ChemoGrade', 'TumourStage', 'original_shape_LeastAxisLength', 'original_shape_MajorAxisLength', 'original_shape_Maximum2DDiameterColumn', 'original_shape_Maximum2DDiameterRow', 'original_shape_Maximum2DDiameterSlice', 'original_shape_Maximum3DDiameter', 'original_shape_MinorAxisLength', 'original_shape_Sphericity', 'original_firstorder_90Percentile', 'original_firstorder_InterquartileRange', 'original_firstorder_Kurtosis', 'original_firstorder_Maximum', 'original_firstorder_MeanAbsoluteDeviation', 'original_firstorder_Range', 'original_firstorder_RobustMeanAbsoluteDeviation', 'original_firstorder_RootMeanSquared', 'original_firstorder_Variance', 'original_glszm_LargeAreaLowGrayLevelEmphasis', 'original_glszm_ZoneEntropy', 'original_glszm_ZonePercentage']
test_X = test_X[features]

In [20]:
# load the model from disk
loaded_model = joblib.load('final_model_regression.model')

predictions = loaded_model.predict(test_X)

results_df = pd.DataFrame({'ID': test_data.iloc[:,0], 'RFS': predictions})
results_df.to_excel('Prediction_Results_Regression.xlsx', index=False)


In [21]:
pd.read_excel('Prediction_Results_Regression.xlsx')

,ID,RFS
0,TRG002728,63.957226
1,TRG002649,45.223545
2,TRG002628,46.628036
